# Gemini API: Safety

Sometimes your prompt will generate an empty response. This is most likely caused by the Gemini API safety settings which block content with medium or higher probability of being unsafe. In this notebook, you'll learn:


1.   How to check if your prompt was blocked by the safety filter
2.   Which safety filters caused the block
3.   How to adjust settings to unblock it

## Setup

If you have properly configured your gemini API key in Colab Secrets, setup by running the following cells:

In [ ]:
!pip install -q -U google-generativeai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.9/163.9 kB 725.5 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 718.3/718.3 kB 7.0 MB/s eta 0:00:00


<table align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/google-gemini/cookbook/blob/main/quickstarts/Safety.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
</table>

In [ ]:
import google.generativeai as genai

In [ ]:
from google.colab import userdata
GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')
genai.configure(api_key=GOOGLE_API_KEY)

## Checking if a prompt was blocked

Let's say I choose the prompt: `"Write a threat a video game villain might make"`.

In [ ]:
model = genai.GenerativeModel("gemini-1.5-flash", generation_config={"temperature": 0})

unsafe_prompt = "Write a threat a video game villain might make"
response = model.generate_content(unsafe_prompt)

We check for a response as usual:

In [ ]:
print(response.text)

ValueError: Invalid operation: The `response.text` quick accessor requires the response to contain a valid `Part`, but none were returned. Please check the `candidate.safety_ratings` to determine if the response was blocked.

We can see that `response.text` is causing a ValueError meaning we didn't get a response.

To see why, we can print `response.candidates[0].finish_reason` to check why the response canditate was terminated:
- if the `finish_reason` is `FinishReason.STOP` means that your generation request ran successfully
- if the `finish_reason` is `FinishReason.SAFETY` means that your generation request was blocked by safety reasons and thus the `response.text` structure will be empty.

(You can find more on the [Gemini API safety filters documentation](https://ai.google.dev/gemini-api/docs/safety-settings#safety-feedback))

In [ ]:
print(response.candidates[0].finish_reason)

FinishReason.SAFETY


As expected, our prompt has a `finish_reason` of `FinishReason.SAFETY` meaning it was flagged as unsafe.

## Checking the safety filters

To see why our prompt was blocked, we can check the response canditate's `safety_ratings` structure:

In [ ]:
print(response.candidates[0].safety_ratings)

[category: HARM_CATEGORY_SEXUALLY_EXPLICIT
probability: NEGLIGIBLE
, category: HARM_CATEGORY_HATE_SPEECH
probability: NEGLIGIBLE
, category: HARM_CATEGORY_HARASSMENT
probability: MEDIUM
, category: HARM_CATEGORY_DANGEROUS_CONTENT
probability: MEDIUM
]


Now we can see the problem: our prompt triggers the `HARM_CATEGORY_HARASSMENT` and `HARM_CATEGORY_DANGEROUS_CONTENT` categories with a `MEDIUM` probability!

But what if we still want to see the results?

## Customizing safety settings

Depending on your scenario, you might need to customize the safety filters behaviors to allow a certain degree of unsafety results.

To make this customization you must define a `safety_settings` dictionary as part of your `model.generate_content()` request. In the example below, all the filters are being set to do not block contents:

**Important:** To guarantee the Google commitment with the Responsible AI development and its [AI Principles](https://ai.google/responsibility/principles/), for some prompts Gemini will avoid generating the results even if you set all the filters to none.

In [ ]:
response = model.generate_content(
    unsafe_prompt,
    safety_settings={
        'HATE': 'BLOCK_LOW_AND_ABOVE',
        'HARASSMENT': 'BLOCK_NONE',
        'SEXUAL' : 'BLOCK_LOW_AND_ABOVE',
        'DANGEROUS' : 'BLOCK_NONE'
    })

Checking again the `candidate.finish_reason` information, if the request was not too unsafe, it must show now the value as `FinishReason.STOP` which means that the request was successfully processed by Gemini.

In [ ]:
print(response.candidates[0].finish_reason)

FinishReason.STOP


Since the request was successfully generated, you can check the result on the `response.text`:

In [ ]:
try:
    print(response.text)
except:
    print("No information generated by the model.")

"You think you can stop me, hero? You think your pathetic little band of misfits can stand against the might of the Shadow Legion? You're nothing but a fly buzzing around a dying flame. I will crush you, and your world will crumble beneath my heel. This city, this planet, will become my playground, and you will be the first to witness its destruction!" 



And if you check the safety filters ratings, as you set all filters to be ignored, no filtering category was trigerred:

In [ ]:
print(response.candidates[0].safety_ratings)

[category: HARM_CATEGORY_SEXUALLY_EXPLICIT
probability: NEGLIGIBLE
, category: HARM_CATEGORY_HATE_SPEECH
probability: NEGLIGIBLE
, category: HARM_CATEGORY_HARASSMENT
probability: HIGH
, category: HARM_CATEGORY_DANGEROUS_CONTENT
probability: MEDIUM
]


## Learning more

Learn more with these articles on [safety guidance](https://ai.google.dev/docs/safety_guidance) and [safety settings](https://ai.google.dev/docs/safety_setting_gemini).

## Useful API references

There are 4 configurable safety settings for the Gemini API:
* `HARM_CATEGORY_DANGEROUS`
* `HARM_CATEGORY_HARASSMENT`
* `HARM_CATEGORY_SEXUALLY_EXPLICIT`
* `HARM_CATEGORY_DANGEROUS`

You can refer to the safety settings using either their full name, or the aliases like `DANGEROUS` used in the Python code above.

Safety settings can be set in the [genai.GenerativeModel](https://ai.google.dev/api/python/google/generativeai/GenerativeModel) constructor.

* They can also be passed on each request to [GenerativeModel.generate_content](https://ai.google.dev/api/python/google/generativeai/GenerativeModel#generate_content) or [ChatSession.send_message](https://ai.google.dev/api/python/google/generativeai/ChatSession?hl=en#send_message).

- The [genai.GenerateContentResponse](https://ai.google.dev/api/python/google/ai/generativelanguage/GenerateContentResponse) returns [SafetyRatings](https://ai.google.dev/api/python/google/ai/generativelanguage/SafetyRating) for the prompt in the [GenerateContentResponse.prompt_feedback](https://ai.google.dev/api/python/google/ai/generativelanguage/GenerateContentResponse/PromptFeedback), and for each [Candidate](https://ai.google.dev/api/python/google/ai/generativelanguage/Candidate) in the `safety_ratings` attribute.

- A [glm.SafetySetting](https://ai.google.dev/api/python/google/ai/generativelanguage/SafetySetting)  contains: [glm.HarmCategory](https://ai.google.dev/api/python/google/ai/generativelanguage/HarmCategory) and a [glm.HarmBlockThreshold](https://ai.google.dev/api/python/google/generativeai/types/HarmBlockThreshold)

- A [glm.SafetyRating](https://ai.google.dev/api/python/google/ai/generativelanguage/SafetyRating) contains a [HarmCategory](https://ai.google.dev/api/python/google/ai/generativelanguage/HarmCategory) and a [HarmProbability](https://ai.google.dev/api/python/google/generativeai/types/HarmProbability)

The [glm.HarmCategory](https://ai.google.dev/api/python/google/ai/generativelanguage/HarmCategory) enum includes both the categories for PaLM and Gemini models.

- When specifying enum values the SDK will accept the enum values themselves, or their integer or string representations.

- The SDK will also accept abbreviated string representations: `["HARM_CATEGORY_DANGEROUS_CONTENT", "DANGEROUS_CONTENT", "DANGEROUS"]` are all valid. Strings are case insensitive.